In [1]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np

print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.16 | packaged by conda-forge | (default, Feb  1 2023, 15:53:35) [MSC v.1929 64 bit (AMD64)]
pandas version: 2.0.3
numpy version: 1.24.3
matplotlib version: 3.7.1
tqdm version: 4.65.0
sktime version: 0.20.1
xgboost version: 1.7.6
seaborn version: 0.12.2
scikit-learn version: 1.2.2
------------------------------------------------------------------------------


# 공통

In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor

pd.set_option('display.max_columns', 30)
train = pd.read_csv('train.csv')
train.drop(['num_date_time'],axis=1,inplace=True)
test = pd.read_csv('test.csv')
test.drop(['num_date_time'],axis=1,inplace=True)
building = pd.read_csv('building_info.csv')
test = pd.read_csv('merge_test_encoding.csv', encoding = "CP949")
train_loc = pd.read_csv('train_location.csv')
train = pd.concat([train,train_loc['location']],axis=1)
train

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),location
0,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,서울
1,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,서울
2,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,서울
3,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,서울
4,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40,서울
...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04,부여
203996,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96,부여
203997,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12,부여
203998,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08,부여


In [3]:
info_series = building.set_index('건물번호')['연면적(m2)']
train['연면적'] = train['건물번호'].map(info_series)
test['연면적'] = test['건물번호'].map(info_series)

info_series = building.set_index('건물번호')['냉방면적(m2)']
train['냉방면적'] = train['건물번호'].map(info_series)
test['냉방면적'] = test['건물번호'].map(info_series)

info_series = building.set_index('건물번호')['태양광용량(kW)']
train['태양광용량'] = train['건물번호'].map(info_series)
test['태양광용량'] = test['건물번호'].map(info_series)

info_series = building.set_index('건물번호')['ESS저장용량(kWh)']
train['ESS저장용량'] = train['건물번호'].map(info_series)
test['ESS저장용량'] = test['건물번호'].map(info_series)

info_series = building.set_index('건물번호')['PCS용량(kW)']
train['PCS용량'] = train['건물번호'].map(info_series)
test['PCS용량'] = test['건물번호'].map(info_series)
test

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),location,일조(hr),일사(MJ/m2),연면적,냉방면적,태양광용량,ESS저장용량,PCS용량
0,1,20220825 00,23.5,0.0,2.2,72,서울,NaN,NaN,110634.00,39570.00,-,-,-
1,1,20220825 01,23.0,0.0,0.9,72,서울,NaN,NaN,110634.00,39570.00,-,-,-
2,1,20220825 02,22.7,0.0,1.5,75,서울,NaN,NaN,110634.00,39570.00,-,-,-
3,1,20220825 03,22.1,0.0,1.3,78,서울,NaN,NaN,110634.00,39570.00,-,-,-
4,1,20220825 04,21.8,0.0,1.0,77,서울,NaN,NaN,110634.00,39570.00,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,부여,0.5,NaN,57497.84,40035.23,-,-,-
16796,100,20220831 20,20.7,0.0,0.4,95,부여,0.0,NaN,57497.84,40035.23,-,-,-
16797,100,20220831 21,20.2,0.0,0.4,98,부여,NaN,NaN,57497.84,40035.23,-,-,-
16798,100,20220831 22,20.1,0.0,1.1,97,부여,NaN,NaN,57497.84,40035.23,-,-,-


In [4]:
train = train.replace('-', np.nan)
test = test.replace('-', np.nan)
test

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),location,일조(hr),일사(MJ/m2),연면적,냉방면적,태양광용량,ESS저장용량,PCS용량
0,1,20220825 00,23.5,0.0,2.2,72,서울,NaN,NaN,110634.00,39570.00,NaN,NaN,NaN
1,1,20220825 01,23.0,0.0,0.9,72,서울,NaN,NaN,110634.00,39570.00,NaN,NaN,NaN
2,1,20220825 02,22.7,0.0,1.5,75,서울,NaN,NaN,110634.00,39570.00,NaN,NaN,NaN
3,1,20220825 03,22.1,0.0,1.3,78,서울,NaN,NaN,110634.00,39570.00,NaN,NaN,NaN
4,1,20220825 04,21.8,0.0,1.0,77,서울,NaN,NaN,110634.00,39570.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,부여,0.5,NaN,57497.84,40035.23,NaN,NaN,NaN
16796,100,20220831 20,20.7,0.0,0.4,95,부여,0.0,NaN,57497.84,40035.23,NaN,NaN,NaN
16797,100,20220831 21,20.2,0.0,0.4,98,부여,NaN,NaN,57497.84,40035.23,NaN,NaN,NaN
16798,100,20220831 22,20.1,0.0,1.1,97,부여,NaN,NaN,57497.84,40035.23,NaN,NaN,NaN


In [5]:
# '풍속(m/s)' 및 '습도(%)' 피처에 대한 NaN 값은 이전 값으로 채워줍니다.
#train['풍속(m/s)'] = train['풍속(m/s)'].fillna(method='ffill')
#train['습도(%)'] = train['습도(%)'].fillna(method='ffill')
train = train.dropna(subset=['풍속(m/s)'])

# '강수량(mm)' 피처의 NaN 값은 0으로 채워줍니다.
train['강수량(mm)'] = train['강수량(mm)'].fillna(0)

train['일조(hr)'] = train['일조(hr)'].fillna(0)
train['일사(MJ/m2)'] = train['일사(MJ/m2)'].fillna(0)

# '풍속(m/s)' 및 '습도(%)' 피처에 대한 NaN 값은 이전 값으로 채워줍니다.
#test['풍속(m/s)'] = test['풍속(m/s)'].fillna(method='ffill')
#test['습도(%)'] = test['습도(%)'].fillna(method='ffill')

# '강수량(mm)' 피처의 NaN 값은 0으로 채워줍니다.
test['강수량(mm)'] = test['강수량(mm)'].fillna(0)

test['일조(hr)'] = test['일조(hr)'].fillna(0)
test['일사(MJ/m2)'] = test['일사(MJ/m2)'].fillna(0)

In [6]:
train['태양광용량'] = train['태양광용량'].fillna(0)
train['ESS저장용량'] = train['ESS저장용량'].fillna(0)
train['PCS용량'] = train['PCS용량'].fillna(0)

test['태양광용량'] = test['태양광용량'].fillna(0)
test['ESS저장용량'] = test['ESS저장용량'].fillna(0)
test['PCS용량'] = test['PCS용량'].fillna(0)
test

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),location,일조(hr),일사(MJ/m2),연면적,냉방면적,태양광용량,ESS저장용량,PCS용량
0,1,20220825 00,23.5,0.0,2.2,72,서울,0.0,0.0,110634.00,39570.00,0,0,0
1,1,20220825 01,23.0,0.0,0.9,72,서울,0.0,0.0,110634.00,39570.00,0,0,0
2,1,20220825 02,22.7,0.0,1.5,75,서울,0.0,0.0,110634.00,39570.00,0,0,0
3,1,20220825 03,22.1,0.0,1.3,78,서울,0.0,0.0,110634.00,39570.00,0,0,0
4,1,20220825 04,21.8,0.0,1.0,77,서울,0.0,0.0,110634.00,39570.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,부여,0.5,0.0,57497.84,40035.23,0,0,0
16796,100,20220831 20,20.7,0.0,0.4,95,부여,0.0,0.0,57497.84,40035.23,0,0,0
16797,100,20220831 21,20.2,0.0,0.4,98,부여,0.0,0.0,57497.84,40035.23,0,0,0
16798,100,20220831 22,20.1,0.0,1.1,97,부여,0.0,0.0,57497.84,40035.23,0,0,0


In [7]:
train['태양광용량'] = train['태양광용량'].astype(float)
train['ESS저장용량'] = train['ESS저장용량'].astype(float)
train['PCS용량'] = train['PCS용량'].astype(float)

test['태양광용량'] = test['태양광용량'].astype(float)
test['ESS저장용량'] = test['ESS저장용량'].astype(float)
test['PCS용량'] = test['PCS용량'].astype(float)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   건물번호       16800 non-null  int64  
 1   일시         16800 non-null  object 
 2   기온(C)      16800 non-null  float64
 3   강수량(mm)    16800 non-null  float64
 4   풍속(m/s)    16800 non-null  float64
 5   습도(%)      16800 non-null  int64  
 6   location   16800 non-null  object 
 7   일조(hr)     16800 non-null  float64
 8   일사(MJ/m2)  16800 non-null  float64
 9   연면적        16800 non-null  float64
 10  냉방면적       16800 non-null  float64
 11  태양광용량      16800 non-null  float64
 12  ESS저장용량    16800 non-null  float64
 13  PCS용량      16800 non-null  float64
dtypes: float64(10), int64(2), object(2)
memory usage: 1.8+ MB


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['location'] = le.fit_transform(train['location'])
test['location'] = le.transform(test['location'])
train

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0
203996,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0
203997,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0
203998,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0


In [11]:

## 변수들을 영문명으로 변경
cols = ['num', 'date_time', 'temp', 'rainy' ,'wind','hum' , 'sun', 'MJ' ,'power', 'location', '연면적', '냉방면적', '태양광용량', 'ESS저장용량', 'PCS용량']
train.columns = cols

# 'date_time' 컬럼을 문자열로 변환
train['date_time'] = train['date_time'].astype(str)

date = pd.to_datetime(train.date_time, format='%Y%m%d %H')
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.isocalendar().week


cols = ['num', 'date_time', 'temp', 'rainy' ,'wind','hum','location','sun', 'MJ' , '연면적', '냉방면적', '태양광용량', 'ESS저장용량', 'PCS용량']
test.columns = cols

# 'date_time' 컬럼을 문자열로 변환
test['date_time'] = test['date_time'].astype(str)

date = pd.to_datetime(test.date_time, format='%Y%m%d %H')
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.isocalendar().week
test

,num,date_time,temp,rainy,wind,hum,location,sun,MJ,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,day,month,week
0,1,20220825 00,23.5,0.0,2.2,72,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,0,3,8,34
1,1,20220825 01,23.0,0.0,0.9,72,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,1,3,8,34
2,1,20220825 02,22.7,0.0,1.5,75,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,2,3,8,34
3,1,20220825 03,22.1,0.0,1.3,78,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,3,3,8,34
4,1,20220825 04,21.8,0.0,1.0,77,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,4,3,8,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,9,0.5,0.0,57497.84,40035.23,0.0,0.0,0.0,19,2,8,35
16796,100,20220831 20,20.7,0.0,0.4,95,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,20,2,8,35
16797,100,20220831 21,20.2,0.0,0.4,98,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,21,2,8,35
16798,100,20220831 22,20.1,0.0,1.1,97,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,22,2,8,35


In [13]:
# train 일조, 복사량을 강수량 이용해서 test에 hour을 groupby해서 mean값 넣기
# Calculate the mean power usage for each 'num' and 'hour' in the train set
'''
mean_power = train.groupby(['num', 'hour'])['sun'].mean().reset_index()
mean_power.rename(columns={'power': 'sun'}, inplace=True)

# Merge this with the test set
test = pd.merge(test, mean_power,  how='left', on=['num', 'hour'])

# Calculate the mean power usage for each 'num' and 'hour' in the train set
mean_power = train.groupby(['num', 'hour'])['MJ'].mean().reset_index()
mean_power.rename(columns={'power': 'MJ'}, inplace=True)

# Merge this with the test set
test = pd.merge(test, mean_power,  how='left', on=['num', 'hour'])

test
'''

"\nmean_power = train.groupby(['num', 'hour'])['sun'].mean().reset_index()\nmean_power.rename(columns={'power': 'sun'}, inplace=True)\n\n# Merge this with the test set\ntest = pd.merge(test, mean_power,  how='left', on=['num', 'hour'])\n\n# Calculate the mean power usage for each 'num' and 'hour' in the train set\nmean_power = train.groupby(['num', 'hour'])['MJ'].mean().reset_index()\nmean_power.rename(columns={'power': 'MJ'}, inplace=True)\n\n# Merge this with the test set\ntest = pd.merge(test, mean_power,  how='left', on=['num', 'hour'])\n\ntest\n"

In [14]:
# Group by 'num', 'day', 'week', 'month'
grouped_df = test.groupby(['num', 'day', 'week', 'month'])

# Apply function to each group
def process_group(group):
    if (group['rainy'] != 0).any():  # if there is any non-zero 'rainy' value in the group
        group['sun'] = 0  # set all 'sun' values to 0
        group['MJ'] = group['MJ'] / 10  # set all 'MJ' values to 1/10 of original
    return group

# Apply the function to each group
test = grouped_df.apply(process_group).reset_index(drop=True)
test

,num,date_time,temp,rainy,wind,hum,location,sun,MJ,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,day,month,week
0,1,20220829 00,22.5,0.0,1.5,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,0,0,8,35
1,1,20220829 01,22.3,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,1,0,8,35
2,1,20220829 02,22.3,0.0,2.6,67,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,2,0,8,35
3,1,20220829 03,22.1,0.0,1.6,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,3,0,8,35
4,1,20220829 04,21.8,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,4,0,8,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,9,0.8,0.0,57497.84,40035.23,0.0,0.0,0.0,19,6,8,34
16796,100,20220828 20,21.9,0.0,0.3,74,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,20,6,8,34
16797,100,20220828 21,20.3,0.0,0.0,84,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,21,6,8,34
16798,100,20220828 22,20.6,0.0,0.1,83,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,22,6,8,34


In [15]:
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)

train['sin_day'] = np.sin(2*np.pi*train.day/7)
train['cos_day'] = np.cos(2*np.pi*train.day/7)

test['sin_day'] = np.sin(2*np.pi*test.day/7)
test['cos_day'] = np.cos(2*np.pi*test.day/7)

### 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2022-08-15'<=train.date_time)&(train.date_time<'2022-08-16'), 'holiday'] = 1

test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test.loc[('2022-08-15'<=test.date_time)&(test.date_time<'2022-08-16'), 'holiday'] = 1

'''
## 건물별, 요일별, 시간별 발전량 평균 넣어주기
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)
tqdm.pandas()
test['day_hour_mean'] = test.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.day == x['day']) & (power_mean.hour == x['hour']) ,'power'].values[0], axis = 1)


## 건물별 시간별 발전량 평균 넣어주기
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)
tqdm.pandas()
test['hour_mean'] = test.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)


## 건물별 시간별 발전량 표준편차 넣어주기
tqdm.pandas()
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)
test['hour_std'] = test.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)
'''
# Calculate mean and std of 'hour' for each unique 'num'
num_hour_mean = train.groupby(['num', 'hour','day'])['power'].mean()
num_hour_std = train.groupby(['num', 'hour','day'])['power'].std()

# Reset the index of our Series to have a DataFrame with 'num' and 'hour' as columns
num_hour_mean = num_hour_mean.reset_index()
num_hour_std = num_hour_std.reset_index()

# Rename the column to 'hour_mean' and 'hour_std'
num_hour_mean.rename(columns={'power': 'num_day_hour_mean'}, inplace=True)
num_hour_std.rename(columns={'power': 'num_day_hour_std'}, inplace=True)

# Merge the calculated mean and std to the original dataframes
train = pd.merge(train, num_hour_mean, how='left', on=['num', 'hour','day'])
train = pd.merge(train, num_hour_std, how='left', on=['num', 'hour','day'])

test = pd.merge(test, num_hour_mean, how='left', on=['num', 'hour','day'])
test = pd.merge(test, num_hour_std, how='left', on=['num', 'hour','day'])


In [17]:
train['week'] = train['week'].astype('int')
test['week'] = test['week'].astype('int')

In [18]:
#불쾌지수 
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32
test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)


#냉방도일
cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs
cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs



In [19]:
def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [21]:
# 새로운 피처를 만들고 0으로 초기화
features = ['건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛', 
            '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트', '호텔및리조트']

for feature in features:
    train[feature] = 0
    test[feature] = 0
    
# num 값 범위에 따라 피처에 1 할당
num_ranges = [(1, 15, '건물기타'), (16, 23, '공공'), (24, 31, '대학교'), 
              (32, 36, '데이터센터'), (37, 44, '백화점및아울렛'),
              (45, 52, '병원'), (53, 60, '상용'), (61, 68, '아파트'),
              (69, 76, '연구소'), (77, 84, '지식산업센터'), 
              (85, 92, '할인마트'), (93, 100, '호텔및리조트')]

for start, end, feature in num_ranges:
    train.loc[(train['num'] >= start) & (train['num'] <= end), feature] = 1
    test.loc[(test['num'] >= start) & (test['num'] <= end), feature] = 1
test

,num,date_time,temp,rainy,wind,hum,location,sun,MJ,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,...,num_day_hour_std,THI,CDH,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트
0,1,20220829 00,22.5,0.0,1.5,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,0,...,503.140848,55.1464,-3.5,1,0,0,0,0,0,0,0,0,0,0,0
1,1,20220829 01,22.3,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,1,...,503.630471,54.7864,-7.2,1,0,0,0,0,0,0,0,0,0,0,0
2,1,20220829 02,22.3,0.0,2.6,67,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,2,...,463.827831,54.9701,-10.9,1,0,0,0,0,0,0,0,0,0,0,0
3,1,20220829 03,22.1,0.0,1.6,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,3,...,439.082379,54.4264,-14.8,1,0,0,0,0,0,0,0,0,0,0,0
4,1,20220829 04,21.8,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,4,...,429.318247,53.8864,-19.0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,9,0.8,0.0,57497.84,40035.23,0.0,0.0,0.0,19,...,159.902858,55.3425,-19.6,0,0,0,0,0,0,0,0,0,0,0,1
16796,100,20220828 20,21.9,0.0,0.3,74,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,20,...,119.116420,56.0904,-15.4,0,0,0,0,0,0,0,0,0,0,0,1
16797,100,20220828 21,20.3,0.0,0.0,84,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,21,...,127.509524,57.5224,-14.6,0,0,0,0,0,0,0,0,0,0,0,1
16798,100,20220828 22,20.6,0.0,0.1,83,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,22,...,96.203972,57.5421,-15.6,0,0,0,0,0,0,0,0,0,0,0,1


In [22]:
train['tem_x_hum'] = train['temp'] * train['hum']
test['tem_x_hum'] = test['temp'] * test['hum']
test

,num,date_time,temp,rainy,wind,hum,location,sun,MJ,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,...,THI,CDH,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum
0,1,20220829 00,22.5,0.0,1.5,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,0,...,55.1464,-3.5,1,0,0,0,0,0,0,0,0,0,0,0,1485.0
1,1,20220829 01,22.3,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,1,...,54.7864,-7.2,1,0,0,0,0,0,0,0,0,0,0,0,1471.8
2,1,20220829 02,22.3,0.0,2.6,67,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,2,...,54.9701,-10.9,1,0,0,0,0,0,0,0,0,0,0,0,1494.1
3,1,20220829 03,22.1,0.0,1.6,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,3,...,54.4264,-14.8,1,0,0,0,0,0,0,0,0,0,0,0,1458.6
4,1,20220829 04,21.8,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,4,...,53.8864,-19.0,1,0,0,0,0,0,0,0,0,0,0,0,1438.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,9,0.8,0.0,57497.84,40035.23,0.0,0.0,0.0,19,...,55.3425,-19.6,0,0,0,0,0,0,0,0,0,0,0,1,1475.5
16796,100,20220828 20,21.9,0.0,0.3,74,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,20,...,56.0904,-15.4,0,0,0,0,0,0,0,0,0,0,0,1,1620.6
16797,100,20220828 21,20.3,0.0,0.0,84,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,21,...,57.5224,-14.6,0,0,0,0,0,0,0,0,0,0,0,1,1705.2
16798,100,20220828 22,20.6,0.0,0.1,83,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,22,...,57.5421,-15.6,0,0,0,0,0,0,0,0,0,0,0,1,1709.8


In [23]:
def calculate_commute_times(df):
    # Calculate the power difference with previous hour
    df['power_diff'] = df.groupby('num')['power'].diff()

    # Calculate the hour which has the maximum positive difference within 6~10 (assuming it's the start hour)
    start_hour = df[(df['power_diff'] > 0) & (df['hour'].between(6,10))].groupby('num')['power_diff'].idxmax().reset_index()
    start_hour.columns = ['num', 'hour_idx']
    start_hour['start_hour'] = df.loc[start_hour['hour_idx'], 'hour'].values
    start_hour = start_hour.drop(columns='hour_idx')

    # Calculate the hour which has the maximum negative difference within 17~22 (assuming it's the end hour)
    end_hour = df[(df['power_diff'] < 0) & (df['hour'].between(17,22))].groupby('num')['power_diff'].idxmin().reset_index()
    end_hour.columns = ['num', 'hour_idx']
    end_hour['end_hour'] = df.loc[end_hour['hour_idx'], 'hour'].values
    end_hour = end_hour.drop(columns='hour_idx')

    # Merge the calculated start and end hours to the original dataframe
    df = df.merge(start_hour, on='num')
    df = df.merge(end_hour, on='num')

    # Drop the used column
    df = df.drop(columns='power_diff')

    return df

train = calculate_commute_times(train)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,...,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,start_hour,end_hour
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0,0,0,781.2,10,18
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0,0,0,810.0,10,18
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0,0,0,796.5,10,18
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0,0,0,801.6,10,18
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0,0,0,791.2,10,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,1986.6,7,17
203977,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,1926.4,7,17
203978,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,1959.6,7,17
203979,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,1974.0,7,17


In [24]:
def calculate_commute_period(df):
    # Get the start hour and end hour for each 'num'
    start_hour = df.groupby('num')['start_hour'].first().reset_index()
    end_hour = df.groupby('num')['end_hour'].first().reset_index()

    # Define a function to apply to the DataFrame
    def is_commute_period(row):
        if row['hour'] >= start_hour.loc[row['num'] - 1, 'start_hour'] and row['hour'] <= end_hour.loc[row['num'] - 1, 'end_hour']:
            return 1
        else:
            return 0

    # Apply the function to each row
    df['commute_period'] = df.apply(is_commute_period, axis=1)

    return df

# Apply the function to the DataFrame
train = calculate_commute_period(train)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,...,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,start_hour,end_hour,commute_period
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,781.2,10,18,0
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,810.0,10,18,0
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,796.5,10,18,0
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,801.6,10,18,0
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,791.2,10,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1986.6,7,17,0
203977,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1926.4,7,17,0
203978,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1959.6,7,17,0
203979,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1974.0,7,17,0


In [25]:
# Calculate the commute time for each 'num' in the training set
num_commute_time_map = train.groupby('num')[['start_hour', 'end_hour']].first().to_dict('index')

# Apply the commute times to the test set
for num, times in num_commute_time_map.items():
    test.loc[test['num'] == num, 'commute_period'] = ((test['hour'] >= times['start_hour']) & (test['hour'] <= times['end_hour'])).astype(int)
train.drop(['start_hour', 'end_hour'],axis=1,inplace=True)
test

,num,date_time,temp,rainy,wind,hum,location,sun,MJ,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,...,CDH,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period
0,1,20220829 00,22.5,0.0,1.5,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,0,...,-3.5,1,0,0,0,0,0,0,0,0,0,0,0,1485.0,0.0
1,1,20220829 01,22.3,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,1,...,-7.2,1,0,0,0,0,0,0,0,0,0,0,0,1471.8,0.0
2,1,20220829 02,22.3,0.0,2.6,67,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,2,...,-10.9,1,0,0,0,0,0,0,0,0,0,0,0,1494.1,0.0
3,1,20220829 03,22.1,0.0,1.6,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,3,...,-14.8,1,0,0,0,0,0,0,0,0,0,0,0,1458.6,0.0
4,1,20220829 04,21.8,0.0,1.3,66,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,4,...,-19.0,1,0,0,0,0,0,0,0,0,0,0,0,1438.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,9,0.8,0.0,57497.84,40035.23,0.0,0.0,0.0,19,...,-19.6,0,0,0,0,0,0,0,0,0,0,0,1,1475.5,0.0
16796,100,20220828 20,21.9,0.0,0.3,74,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,20,...,-15.4,0,0,0,0,0,0,0,0,0,0,0,1,1620.6,0.0
16797,100,20220828 21,20.3,0.0,0.0,84,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,21,...,-14.6,0,0,0,0,0,0,0,0,0,0,0,1,1705.2,0.0
16798,100,20220828 22,20.6,0.0,0.1,83,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,22,...,-15.6,0,0,0,0,0,0,0,0,0,0,0,1,1709.8,0.0


In [26]:
test = test.copy()
test = test.sort_values(by=['num', 'date_time'])
test = test.reset_index(drop = True)
test

,num,date_time,temp,rainy,wind,hum,location,sun,MJ,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,...,CDH,건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period
0,1,20220825 00,23.5,0.0,2.2,72,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,0,...,-35.3,1,0,0,0,0,0,0,0,0,0,0,0,1692.0,0.0
1,1,20220825 01,23.0,0.0,0.9,72,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,1,...,-35.5,1,0,0,0,0,0,0,0,0,0,0,0,1656.0,0.0
2,1,20220825 02,22.7,0.0,1.5,75,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,2,...,-36.4,1,0,0,0,0,0,0,0,0,0,0,0,1702.5,0.0
3,1,20220825 03,22.1,0.0,1.3,78,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,3,...,-37.9,1,0,0,0,0,0,0,0,0,0,0,0,1723.8,0.0
4,1,20220825 04,21.8,0.0,1.0,77,11,0.0,0.0,110634.00,39570.00,0.0,0.0,0.0,4,...,-39.8,1,0,0,0,0,0,0,0,0,0,0,0,1678.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,19,...,-34.5,0,0,0,0,0,0,0,0,0,0,0,1,1890.0,0.0
16796,100,20220831 20,20.7,0.0,0.4,95,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,20,...,-34.4,0,0,0,0,0,0,0,0,0,0,0,1,1966.5,0.0
16797,100,20220831 21,20.2,0.0,0.4,98,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,21,...,-35.3,0,0,0,0,0,0,0,0,0,0,0,1,1979.6,0.0
16798,100,20220831 22,20.1,0.0,1.1,97,9,0.0,0.0,57497.84,40035.23,0.0,0.0,0.0,22,...,-36.8,0,0,0,0,0,0,0,0,0,0,0,1,1949.7,0.0


## 718 New

In [27]:
def assign_THI_group(val):
    if val < 75:
        return 0
    elif 75 <= val < 100:
        return 1
    else:
        return None  # If there are any THI values outside the specified ranges

# Apply the function to the 'THI' column to create a new feature
train['THI_group'] = train['THI'].apply(assign_THI_group)
test['THI_group'] = test['THI'].apply(assign_THI_group)

#체감온도
def body_temp(val):
    if val < 21:
        return 0
    elif 21 <= val < 25:
        return 1
    elif 25 <= val < 28:
        return 2
    elif 28 <= val < 31:
        return 3
    elif 31 <= val:
        return 4
    else:
        return None  # If there are any THI values outside the specified ranges
train['body_temp'] = train['temp'].apply(body_temp)
test['body_temp'] = test['temp'].apply(body_temp)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,...,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,THI_group,body_temp
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,781.2,0,0,0
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,810.0,0,0,0
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,796.5,0,0,0
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,801.6,0,0,0
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,791.2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1986.6,0,0,1
203977,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1926.4,0,0,1
203978,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1959.6,0,0,1
203979,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,1,1974.0,0,0,1


In [28]:
# 일단 모든 값을 0으로 초기화
train['low_power_day'] = 0

# 건물기타
train.loc[(train['num'].isin([2, 3]) & (train['day'] == 0)) | ((train['num'] == 5) & (train['day'].between(0,3))), 'low_power_day'] = 1

# 공공
train.loc[(train['num'].isin([17, 18, 19, 20, 21, 22, 23])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1

# 대학교
train.loc[(train['num'].isin([24, 25, 26, 27, 28, 29, 30, 31])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1

# 병원
train.loc[(train['num'].isin([45, 50]) & (train['day'] == 5)), 'low_power_day'] = 0.5
train.loc[(train['num'].isin([45, 50]) & (train['day'] == 6)), 'low_power_day'] = 1
train.loc[(train['num'].isin([46, 47, 48, 49, 51, 52])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1

# 상용
train.loc[((train['num'].isin([53, 55, 57, 58, 59, 60]) & (train['day'].isin([5, 6]))) | ((train['num'] == 54) & (train['day'] == 0))), 'low_power_day'] = 1

# 연구소
train.loc[(train['num'].isin([69, 70, 71, 72, 73, 74, 76]) & (train['day'].isin([5, 6]))), 'low_power_day'] = 1

# 지식산업센터
train.loc[(train['num'].isin([77, 78, 79, 80, 82, 83, 84])) & (train['day'].isin([5, 6])), 'low_power_day'] = 1
# 일단 모든 값을 0으로 초기화
test['low_power_day'] = 0

# 건물기타
test.loc[(test['num'].isin([2, 3]) & (test['day'] == 0)) | ((test['num'] == 5) & (test['day'].between(0,3))), 'low_power_day'] = 1

# 공공
test.loc[(test['num'].isin([17, 18, 19, 20, 21, 22, 23])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1

# 대학교
test.loc[(test['num'].isin([24, 25, 26, 27, 28, 29, 30, 31])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1

# 병원
test.loc[(test['num'].isin([45, 50]) & (test['day'] == 5)), 'low_power_day'] = 0.5
test.loc[(test['num'].isin([45, 50]) & (test['day'] == 6)), 'low_power_day'] = 1
test.loc[(test['num'].isin([46, 47, 48, 49, 51, 52])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1

# 상용
test.loc[((test['num'].isin([53, 55, 57, 58, 59, 60]) & (test['day'].isin([5, 6]))) | ((test['num'] == 54) & (test['day'] == 0))), 'low_power_day'] = 1

# 연구소
test.loc[(test['num'].isin([69, 70, 71, 72, 73, 74, 76]) & (test['day'].isin([5, 6]))), 'low_power_day'] = 1

# 지식산업센터
test.loc[(test['num'].isin([77, 78, 79, 80, 82, 83, 84])) & (test['day'].isin([5, 6])), 'low_power_day'] = 1
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,...,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,THI_group,body_temp,low_power_day
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,781.2,0,0,0,0.0
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,810.0,0,0,0,0.0
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,796.5,0,0,0,0.0
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,801.6,0,0,0,0.0
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,791.2,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1,1986.6,0,0,1,0.0
203977,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1,1926.4,0,0,1,0.0
203978,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1,1959.6,0,0,1,0.0
203979,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1,1974.0,0,0,1,0.0


In [29]:
power_diff_ratio_dict = {}

for num in range(1, 101): 
    low_power_day_mean_power = train[(train['num'] == num) & (train['low_power_day'] == 1)]['power'].mean()
    not_low_power_day_mean_power = train[(train['num'] == num) & (train['low_power_day'] == 0)]['power'].mean()
    ambiguous_power_day_mean_power = train[(train['num'] == num) & (train['low_power_day'] == 0.5)]['power'].mean()
    
    if pd.isnull(low_power_day_mean_power) or pd.isnull(not_low_power_day_mean_power):
        power_diff_ratio = 0
    else:
        # Add the half of the ambiguous power day mean to both the low power and not low power days mean
        low_power_day_mean_power = (low_power_day_mean_power + ambiguous_power_day_mean_power / 2) if not pd.isnull(ambiguous_power_day_mean_power) else low_power_day_mean_power
        not_low_power_day_mean_power = (not_low_power_day_mean_power + ambiguous_power_day_mean_power / 2) if not pd.isnull(ambiguous_power_day_mean_power) else not_low_power_day_mean_power

        # Calculate the difference ratio
        power_diff_ratio = abs(low_power_day_mean_power - not_low_power_day_mean_power) / not_low_power_day_mean_power
    
    power_diff_ratio_dict[num] = power_diff_ratio

train['power_diff_ratio'] = train['num'].map(power_diff_ratio_dict)
test['power_diff_ratio'] = test['num'].map(power_diff_ratio_dict)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,...,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,THI_group,body_temp,low_power_day,power_diff_ratio
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,781.2,0,0,0,0.0,0.0
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,810.0,0,0,0,0.0,0.0
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,796.5,0,0,0,0.0,0.0
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,801.6,0,0,0,0.0,0.0
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,791.2,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,1986.6,0,0,1,0.0,0.0
203977,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,1926.4,0,0,1,0.0,0.0
203978,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,1959.6,0,0,1,0.0,0.0
203979,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,1974.0,0,0,1,0.0,0.0


In [30]:
# 32번, 33번만 해당하는 피처
train['power_increase_summer'] = 0
test['power_increase_summer'] = 0

# 7, 8월에 해당하는 32, 33번 num에 대해 'power_increase_summer' 값을 1로 설정
train.loc[(train['num'].isin([32, 33])) & (train['month'].isin([7, 8])), 'power_increase_summer'] = 1
test.loc[(test['num'].isin([32, 33])) & (test['month'].isin([7, 8])), 'power_increase_summer'] = 1

In [31]:
print("실제값이 100일 때 50으로 underestimate할 때의 SMAPE : {}".format(SMAPE(3000, 2820)))
print("실제값이 100일 때 150으로 overestimate할 때의 SMAPE : {}".format(SMAPE(3000, 3500)))
print("실제값이 100일 때 50으로 underestimate할 때의 SMAPE : {}".format(SMAPE(3000, 2750)))
print("실제값이 100일 때 150으로 overestimate할 때의 SMAPE : {}".format(SMAPE(3000, 3850)))
#전체 num 한번에 모델링 할거면, 그루핑, 온도 min, max 값, 

실제값이 100일 때 50으로 underestimate할 때의 SMAPE : 6.185567010309279
실제값이 100일 때 150으로 overestimate할 때의 SMAPE : 15.384615384615385
실제값이 100일 때 50으로 underestimate할 때의 SMAPE : 8.695652173913043
실제값이 100일 때 150으로 overestimate할 때의 SMAPE : 24.817518248175183


In [32]:
train.drop(['sun', 'MJ'],axis=1,inplace=True)
test.drop(['sun','MJ'],axis=1,inplace=True)

In [33]:
train['log1p_rainy'] = np.log1p(train['rainy'])
train['log1p_wind'] = np.log1p(train['wind'])
train['log1p_temp'] = np.log1p(train['temp'])
train['log1p_hum'] = np.log1p(train['hum'])

test['log1p_rainy'] = np.log1p(test['rainy'])
test['log1p_wind'] = np.log1p(test['wind'])
test['log1p_temp'] = np.log1p(test['temp'])
test['log1p_hum'] = np.log1p(test['hum'])
train

,num,date_time,temp,rainy,wind,hum,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,day,...,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,THI_group,body_temp,low_power_day,power_diff_ratio,power_increase_summer,log1p_rainy,log1p_wind,log1p_temp,log1p_hum
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,781.2,0,0,0,0.0,0.0,0,0.0,0.641854,2.975530,3.761200
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,1,2,...,0,0,0,0,810.0,0,0,0,0.0,0.0,0,0.0,0.741937,2.944439,3.828641
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,2,2,...,0,0,0,0,796.5,0,0,0,0.0,0.0,0,0.0,0.916291,2.928524,3.828641
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,3,2,...,0,0,0,0,801.6,0,0,0,0.0,0.0,0,0.0,0.875469,2.873565,3.891820
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,4,2,...,0,0,0,0,791.2,0,0,0,0.0,0.0,0,0.0,1.335001,2.965273,3.784190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,19,2,...,0,0,0,1,1986.6,0,0,1,0.0,0.0,0,0.0,0.641854,3.182212,4.465908
203977,100,20220824 20,22.4,0.0,1.3,86.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,20,2,...,0,0,0,1,1926.4,0,0,1,0.0,0.0,0,0.0,0.832909,3.152736,4.465908
203978,100,20220824 21,21.3,0.0,1.0,92.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,21,2,...,0,0,0,1,1959.6,0,0,1,0.0,0.0,0,0.0,0.693147,3.104587,4.532599
203979,100,20220824 22,21.0,0.0,0.3,94.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,22,2,...,0,0,0,1,1974.0,0,0,1,0.0,0.0,0,0.0,0.262364,3.091042,4.553877


In [34]:
train

,num,date_time,temp,rainy,wind,hum,power,location,연면적,냉방면적,태양광용량,ESS저장용량,PCS용량,hour,day,...,연구소,지식산업센터,할인마트,호텔및리조트,tem_x_hum,commute_period,THI_group,body_temp,low_power_day,power_diff_ratio,power_increase_summer,log1p_rainy,log1p_wind,log1p_temp,log1p_hum
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,11,110634.00,39570.00,0.0,0.0,0.0,0,2,...,0,0,0,0,781.2,0,0,0,0.0,0.0,0,0.0,0.641854,2.975530,3.761200
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,11,110634.00,39570.00,0.0,0.0,0.0,1,2,...,0,0,0,0,810.0,0,0,0,0.0,0.0,0,0.0,0.741937,2.944439,3.828641
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,11,110634.00,39570.00,0.0,0.0,0.0,2,2,...,0,0,0,0,796.5,0,0,0,0.0,0.0,0,0.0,0.916291,2.928524,3.828641
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,11,110634.00,39570.00,0.0,0.0,0.0,3,2,...,0,0,0,0,801.6,0,0,0,0.0,0.0,0,0.0,0.875469,2.873565,3.891820
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,11,110634.00,39570.00,0.0,0.0,0.0,4,2,...,0,0,0,0,791.2,0,0,0,0.0,0.0,0,0.0,1.335001,2.965273,3.784190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203976,100,20220824 19,23.1,0.0,0.9,86.0,881.04,9,57497.84,40035.23,0.0,0.0,0.0,19,2,...,0,0,0,1,1986.6,0,0,1,0.0,0.0,0,0.0,0.641854,3.182212,4.465908
203977,100,20220824 20,22.4,0.0,1.3,86.0,798.96,9,57497.84,40035.23,0.0,0.0,0.0,20,2,...,0,0,0,1,1926.4,0,0,1,0.0,0.0,0,0.0,0.832909,3.152736,4.465908
203978,100,20220824 21,21.3,0.0,1.0,92.0,825.12,9,57497.84,40035.23,0.0,0.0,0.0,21,2,...,0,0,0,1,1959.6,0,0,1,0.0,0.0,0,0.0,0.693147,3.104587,4.532599
203979,100,20220824 22,21.0,0.0,0.3,94.0,640.08,9,57497.84,40035.23,0.0,0.0,0.0,22,2,...,0,0,0,1,1974.0,0,0,1,0.0,0.0,0,0.0,0.262364,3.091042,4.553877


## XGB

In [35]:

def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [36]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def objective_xgb(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": trial.suggest_int('n_estimators', 500, 5000),
        'max_depth': trial.suggest_int('max_depth', 8, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'gamma': trial.suggest_int('gamma', 1, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        #'alpha': trial.suggest_loguniform('alpha', 1e-3, 100.0),
        'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
        'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
        'random_state': 724
    }

    model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0, seed=724)
    alpha_value = model.get_params()['alpha']
    model.set_params(**{'objective':weighted_mse(alpha_value)})
    model.fit(X_train, y_train, verbose = False, eval_set=[(X_val, y_val)], early_stopping_rounds=50)
    y_pred = model.predict(X_val)
    score = SMAPE(y_val, y_pred)

    return score


### fit_public

In [38]:
#나중엔 week로 총 13? 14폴드 박기
preds = np.array([]) 
best_params = {}
best_values = {}  # New dictionary to store the best values
# Select the data for the current num
num_data = train.copy()

# Combine 'hour', 'day', 'week' to a new feature as stratified target
#num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str)

# Split the data into training and validation set
train_df, val_df = train_test_split(num_data, test_size=0.2, stratify=num_data['stratified_target'], random_state=724)

# Drop the temporary feature
train_df = train_df.drop(columns='stratified_target')
val_df = val_df.drop(columns='stratified_target')

# Validation set에서 day 값이 3, 4, 5이 아닌 행을 training set에 추가
train_df = pd.concat([train_df, val_df[~val_df['day'].isin([3, 4, 5])]])

# Validation set에서 day 값이 3, 4, 5인 행만 남기기
val_df = val_df[val_df['day'].isin([3, 4, 5])]

y_train = train_df['power']
y_val = val_df['power']

x_train, x_test = train_df.drop(['date_time'],axis=1), test.drop(['date_time'],axis=1)
x_val = val_df.drop(['date_time'],axis=1)

x_train.drop(['power'],axis=1,inplace=True)
x_val.drop(['power'],axis=1,inplace=True)

x_test = x_test[x_train.columns]

param = {'n_estimators': 4387, 'max_depth': 13, 'min_child_weight': 6, 'gamma': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.6816991771444707, 'lambda': 0.013747157221037776, 'alpha': 1.5291313428196907, 'subsample': 0.7}

xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=724)
alpha_value2 = xgb.get_params()['alpha']
xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
##근데 저렇게 validation 하면 학습 셋도 좀 이상해지는데 이게 맞나?0.4
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
preds = np.append(preds, y_pred)
preds


array([1996.14611816, 1936.09802246, 1688.31774902, ...,  731.0614624 ,
        649.97735596,  544.29205322])

In [37]:
#나중엔 week로 총 13? 14폴드 박기
preds = np.array([]) 
best_params = {}
best_values = {}  # New dictionary to store the best values
with open('best_params_values_0724.txt', 'w') as f:
    # Select the data for the current num
    num_data = train.copy()

    # Combine 'hour', 'day', 'week' to a new feature as stratified target
    #num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str) + '_' + num_data['week'].astype(str) + '_' + num_data['month'].astype(str)
    num_data['stratified_target'] = num_data['hour'].astype(str) + '_' + num_data['day'].astype(str)

    # Split the data into training and validation set
    train_df, val_df = train_test_split(num_data, test_size=0.2, stratify=num_data['stratified_target'], random_state=724)

    # Drop the temporary feature
    train_df = train_df.drop(columns='stratified_target')
    val_df = val_df.drop(columns='stratified_target')

    # Validation set에서 day 값이 3, 4, 5이 아닌 행을 training set에 추가
    train_df = pd.concat([train_df, val_df[~val_df['day'].isin([3, 4, 5])]])

    # Validation set에서 day 값이 3, 4, 5인 행만 남기기
    val_df = val_df[val_df['day'].isin([3, 4, 5])]

    y_train = train_df['power']
    y_val = val_df['power']

    x_train, x_test = train_df.drop(['date_time'],axis=1), test.drop(['date_time'],axis=1)
    x_val = val_df.drop(['date_time'],axis=1)

    x_train.drop(['power'],axis=1,inplace=True)
    x_val.drop(['power'],axis=1,inplace=True)

    x_test = x_test[x_train.columns]

    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_val, y_val), n_trials=200)
    param = study.best_trial.params
    best_params = param
    best_values = study.best_trial.value  # Store the best value
    f.write(f'Best Params: {best_params}, \nBest Values: {best_values}\n\n')
    f.flush()
    xgb = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0, seed=724)
    alpha_value2 = xgb.get_params()['alpha']
    xgb.set_params(**{'objective':weighted_mse(alpha_value2)})
    ##근데 저렇게 validation 하면 학습 셋도 좀 이상해지는데 이게 맞나?0.4
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    preds = np.append(preds, y_pred)


[I 2023-07-24 00:16:29,433] A new study created in memory with name: no-name-5e121536-4257-466e-9a38-f9eb848a8c95
C:\Users\ineeji\AppData\Local\Temp\ipykernel_55804\2248152643.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\ineeji\AppData\Local\Temp\ipykernel_55804\2248152643.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1, 100.0),
c:\Users\ineeji\anaconda3\envs\v1\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in cons

KeyboardInterrupt: 

### 유독 SMAPE값이 엄청 낮은 빌딩 종류들이 존재. 이유 파악해서 모델링

### 타겟값을 면적으로 나눠볼까?

In [41]:
submission = pd.read_csv('sample_submission.csv')
submission['answer'] = preds
submission.to_csv('submit_v16_xgb_724.csv', index = False)

In [42]:
submission 

,num_date_time,answer
0,1_20220825 00,1996.146118
1,1_20220825 01,1936.098022
2,1_20220825 02,1688.317749
3,1_20220825 03,1570.802612
4,1_20220825 04,1578.209473
...,...,...
16795,100_20220831 19,908.076538
16796,100_20220831 20,813.324280
16797,100_20220831 21,731.061462
16798,100_20220831 22,649.977356


In [179]:
submission

,num_date_time,answer
0,1_20220825 00,1957.390503
1,1_20220825 01,1916.372314
2,1_20220825 02,1770.732666
3,1_20220825 03,1652.366577
4,1_20220825 04,1658.233154
...,...,...
16795,100_20220831 19,906.117065
16796,100_20220831 20,835.492188
16797,100_20220831 21,773.684326
16798,100_20220831 22,652.045593


# building별 Modeling

# 전체 한번에 Modeling

# ens

In [34]:
a = pd.read_csv('submission2.csv')
b = pd.read_csv('submit_v12_xgb_optuna_iter41_8888.csv')
c = a.copy()
c['answer'] = 0
c['answer'] = a['answer']*0.42185 + b['answer']*0.57815
#c['answer'] = (a['answer'] + b['answer']) / 2
c.to_csv('submit_v13_xgb_optuna_8512.csv',index=False)
c

,num_date_time,answer
0,1_20220825 00,2032.590085
1,1_20220825 01,2001.954761
2,1_20220825 02,1804.802206
3,1_20220825 03,1708.447136
4,1_20220825 04,1699.192685
...,...,...
16795,100_20220831 19,943.590352
16796,100_20220831 20,848.670576
16797,100_20220831 21,764.608321
16798,100_20220831 22,661.657046


In [81]:
a = pd.read_csv('11_12_xgb_41_720_8888_ens.csv')
a['answer'] = a['answer'] * 1.05
a.to_csv('sotapp.csv',index=False)
a

,num_date_time,answer
0,1_20220825 00,2201.706445
1,1_20220825 01,2157.875354
2,1_20220825 02,1942.385477
3,1_20220825 03,1818.851477
4,1_20220825 04,1827.483920
...,...,...
16795,100_20220831 19,923.047476
16796,100_20220831 20,850.975172
16797,100_20220831 21,780.236613
16798,100_20220831 22,650.756886


# post processing

weighted mse와 같은 맥락에서, 과도한 underestimate를 막기 위해 예측값을 후처리했습니다.
- 예측 주로부터 직전 4주(train set 마지막 28일)의 건물별 요일별 시간대별 전력소비량의 최솟값을 구한 뒤,
- test set의 같은 건물 요일 시간대의 예측값과 비교하여 만약 1번의 최솟값보다 예측값이 작다면 최솟값으로 예측값을 대체해주었습니다.
- public score 0.01 , private score 0.08 정도의 성능 향상이 있었습니다.

In [ ]:
train_to_post = pd.read_csv('./data/train.csv', encoding = 'cp949')
cols = ['num', 'date_time', 'power', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']
train_to_post.columns = cols
date = pd.to_datetime(train_to_post.date_time)
train_to_post['hour'] = date.dt.hour
train_to_post['day'] = date.dt.weekday
train_to_post['month'] = date.dt.month
train_to_post['week'] = date.dt.weekofyear
train_to_post = train_to_post.loc[(('2020-08-17'>train_to_post.date_time)|(train_to_post.date_time>='2020-08-18')), ].reset_index(drop = True)

pred_clip = []
test_to_post = pd.read_csv('./data/test.csv',  encoding = 'cp949')
cols = ['num', 'date_time', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']
test_to_post.columns = cols
date = pd.to_datetime(test_to_post.date_time)
test_to_post['hour'] = date.dt.hour
test_to_post['day'] = date.dt.weekday
test_to_post['month'] = date.dt.month
test_to_post['week'] = date.dt.weekofyear

## submission 불러오기
df = pd.read_csv('./submission/submission_xgb_noclip.csv')
for i in range(60):
    min_data = train_to_post.loc[train_to_post.num == i+1, ].iloc[-28*24:, :] ## 건물별로 직전 28일의 데이터 불러오기
    ## 요일별, 시간대별 최솟값 계산
    min_data = pd.pivot_table(min_data, values = 'power', index = ['day', 'hour'], aggfunc = min).reset_index() 
    pred = df.answer[168*i:168*(i+1)].reset_index(drop=True) ## 168개 데이터, 즉 건물별 예측값 불러오기
    day =  test_to_post.day[168*i:168*(i+1)].reset_index(drop=True) ## 예측값 요일 불러오기
    hour = test_to_post.hour[168*i:168*(i+1)].reset_index(drop=True) ## 예측값 시간 불러오기
    df_pred = pd.concat([pred, day, hour], axis = 1)
    df_pred.columns = ['pred', 'day', 'hour']
    for j in range(len(df_pred)):
        min_power = min_data.loc[(min_data.day == df_pred.day[j])&(min_data.hour == df_pred.hour[j]), 'power'].values[0]
        if df_pred.pred[j] < min_power:
            pred_clip.append(min_power)
        else:
            pred_clip.append(df_pred.pred[j])

초록색으로 표시된 값이 원래의 예측값, 주황색이 후처리된 예측값입니다.
변동이 거의 없는 건물도 있으나, 유의미하게 바뀐 건물도 확인됩니다.

In [ ]:
pred_origin = df.answer
pred_clip = pd.Series(pred_clip)

for i in range(60):
    power = train_to_post.loc[train_to_post.num == i+1, 'power'].reset_index(drop=True)
    preds = pred_clip[i*168:(i+1)*168]
    preds_origin = pred_origin[i*168:(i+1)*168]
    preds.index = range(power.index[-1], power.index[-1]+168)
    preds_origin.index = range(power.index[-1], power.index[-1]+168)
    
    plot_series(power, preds,  preds_origin, markers = [',', ',', ','])

In [83]:
building_day_mapping = {
    1 : ([0, 1, 2, 3, 4, 5], [6]),
    2 : ([0], [1, 2, 3, 4, 5, 6]),
    3 : ([0], [1, 2, 3, 4, 5, 6]),
    4 : ([0, 1, 2, 3, 4, 5, 6], []),
    5 : ([0, 1, 2, 3], [4, 5, 6]),
    6 : ([0, 1, 2, 3, 4, 5, 6], []),
    7 : ([0, 1, 2, 3, 4, 5, 6], []),
    8 : ([0, 1, 2, 3, 4, 5], [6]),
    9 : ([0, 1, 2, 3, 4, 5, 6], []),
    10 : ([0, 1, 2, 3, 4, 5, 6], []),
    11 : ([0, 1, 2, 3, 4, 5, 6], []),
    12 : ([0, 1, 2, 3, 4, 5, 6], []),
    13 : ([0, 1, 2, 3, 4, 5, 6], []),
    14 : ([0, 1, 2, 3, 4, 5, 6], []),
    15 : ([0, 1, 2, 3, 4, 5], [6]),
    16 : ([0, 1, 2, 3, 4, 5, 6], []),
    17 : ([0, 1, 2, 3, 4], [5, 6]),
    18 : ([0, 1, 2, 3, 4], [5, 6]),
    19 : ([0, 1, 2, 3, 4], [5, 6]),
    20 : ([0, 1, 2, 3, 4], [5, 6]),
    21 : ([0, 1, 2, 3, 4], [5, 6]),
    22 : ([0, 1, 2, 3, 4], [5, 6]),
    23 : ([0, 1, 2, 3, 4], [5, 6]),
    24 : ([0, 1, 2, 3, 4], [5, 6]),
    25 : ([0, 1, 2, 3, 4], [5, 6]),
    26 : ([0, 1, 2, 3, 4], [5, 6]),
    27 : ([0, 1, 2, 3, 4], [5, 6]),
    28 : ([0, 1, 2, 3, 4], [5, 6]),
    29 : ([0, 1, 2, 3, 4], [5, 6]),
    30 : ([0, 1, 2, 3, 4], [5, 6]),
    31 : ([0, 1, 2, 3, 4], [5, 6]),
    32 : ([0, 1, 2, 3, 4, 5, 6], []),
    33 : ([0, 1, 2, 3, 4, 5, 6], []),
    34 : ([0, 1, 2, 3, 4, 5, 6], []),
    35 : ([0, 1, 2, 3, 4, 5, 6], []),
    36 : ([0, 1, 2, 3, 4, 5, 6], []),
    37 : ([0, 1, 2, 3, 4, 5, 6], []),
    38 : ([0, 1, 2, 3, 4, 5, 6], []),
    39 : ([0, 1, 2, 3, 4, 5, 6], []),
    40 : ([0, 1, 2, 3, 4, 5, 6], []),
    41 : ([0, 1, 2, 3, 4, 5, 6], []),
    42 : ([0, 1, 2, 3, 4, 5, 6], []),
    43 : ([0, 1, 2, 3, 4, 5, 6], []),
    44 : ([0, 1, 2, 3, 4, 5, 6], []),
    45 : ([0, 1, 2, 3, 4], [5, 6]),
    46 : ([0, 1, 2, 3, 4], [5, 6]),
    47 : ([0, 1, 2, 3, 4], [5, 6]),
    48 : ([0, 1, 2, 3, 4], [5, 6]),
    49 : ([0, 1, 2, 3, 4], [5, 6]),
    50 : ([0, 1, 2, 3, 4], [5, 6]),
    51 : ([0, 1, 2, 3, 4], [5, 6]),
    52 : ([0, 1, 2, 3, 4], [5, 6]),
    53 : ([0, 1, 2, 3, 4], [5, 6]),
    54 : ([0] , [1, 2, 3, 4, 5, 6]),
    55 : ([0, 1, 2, 3, 4], [5, 6]),
    56 : ([0, 1, 2, 3, 4], [5, 6]),
    57 : ([0, 1, 2, 3, 4], [5, 6]),
    58 : ([0, 1, 2, 3, 4], [5, 6]),
    59 : ([0, 1, 2, 3, 4], [5, 6]),
    60 : ([0, 1, 2, 3, 4], [5, 6]),
    61 : ([0, 1, 2, 3, 4, 5, 6], []),
    62 : ([0, 1, 2, 3, 4, 5, 6], []),
    63 : ([0, 1, 2, 3, 4, 5, 6], []),
    64 : ([0, 1, 2, 3, 4, 5, 6], []),
    65 : ([0, 1, 2, 3, 4, 5, 6], []),
    66 : ([0, 1, 2, 3, 4, 5, 6], []),
    67 : ([0, 1, 2, 3, 4, 5, 6], []),
    68 : ([0, 1, 2, 3, 4, 5, 6], []),
    69 : ([0, 1, 2, 3, 4], [5, 6]),
    70 : ([0, 1, 2, 3, 4], [5, 6]),
    71 : ([0, 1, 2, 3, 4], [5, 6]),
    72 : ([0, 1, 2, 3, 4], [5, 6]),
    73 : ([0, 1, 2, 3, 4], [5, 6]),
    74 : ([0, 1, 2, 3, 4], [5, 6]),
    75 : ([0, 1, 2, 3, 4, 5, 6], []),
    76 : ([0, 1, 2, 3, 4], [5, 6]),
    77 : ([0, 1, 2, 3, 4], [5, 6]),
    78 : ([0, 1, 2, 3, 4], [5, 6]),
    79 : ([0, 1, 2, 3, 4], [5, 6]),
    80 : ([0, 1, 2, 3, 4], [5, 6]),
    81 : ([0, 1, 2, 3, 4, 5, 6], []),
    82 : ([0, 1, 2, 3, 4], [5, 6]),
    83 : ([0, 1, 2, 3, 4], [5, 6]),
    84 : ([0, 1, 2, 3, 4], [5, 6]),
    85 : ([0, 1, 2, 3, 4, 5], [6]),
    86 : ([0, 1, 2, 3, 4, 5], [6]),
    87 : ([0, 1, 2, 3, 4, 5], [6]),
    88 : ([0, 1, 2, 3, 4, 5], [6]),
    89 : ([0, 1, 2, 3, 4, 5], [6]),
    90 : ([0, 1, 2, 3, 4, 5], [6]),
    91 : ([0, 1, 2, 3, 4, 5], [6]),
    92 : ([0, 1, 2, 3, 4, 5], [6]),
    93 : ([0, 1, 2, 3, 4, 5, 6], []),
    94 : ([0, 1, 2, 3, 4, 5, 6], []),
    95 : ([0, 1, 2, 3, 4, 5, 6], []),
    96 : ([0, 1, 2, 3, 4, 5, 6], []),
    97 : ([0, 1, 2, 3, 4, 5, 6], []),
    98 : ([0, 1, 2, 3, 4, 5, 6], []),
    99 : ([0, 1, 2, 3, 4, 5, 6], []),
    100 : ([0, 1, 2, 3, 4, 5, 6], [])
}

In [84]:
non_seven_days_buildings = []

for building, days in building_day_mapping.items():
    total_days = len(days[0]) + len(days[1])
    if total_days != 7:
        non_seven_days_buildings.append(building)

print(non_seven_days_buildings)


[]


In [97]:
string

"Iteration: 1, \nBest Params: {'n_estimators': 4589, 'max_depth': 14, 'min_child_weight': 45, 'gamma': 3, 'learning_rate': 0.02, 'colsample_bytree': 0.9359164821807846, 'lambda': 0.0010054098181390555, 'alpha': 1.7378519299568984, 'subsample': 0.8}, \nBest Values: 3.309705832658706\n\nIteration: 2, \nBest Params: {'n_estimators': 3241, 'max_depth': 13, 'min_child_weight': 52, 'gamma': 1, 'learning_rate': 0.01, 'colsample_bytree': 0.6078452885067815, 'lambda': 0.025670983179185983, 'alpha': 97.05957083125854, 'subsample': 0.7}, \nBest Values: 6.319837537676198\n\nIteration: 3, \nBest Params: {'n_estimators': 2693, 'max_depth': 8, 'min_child_weight': 15, 'gamma': 3, 'learning_rate': 0.016, 'colsample_bytree': 0.5706311076956346, 'lambda': 5.657518503642933, 'alpha': 1.2602756238240975, 'subsample': 1.0}, \nBest Values: 6.023522227890542\n\nIteration: 4, \nBest Params: {'n_estimators': 2235, 'max_depth': 11, 'min_child_weight': 6, 'gamma': 1, 'learning_rate': 0.012, 'colsample_bytree': 0.

# 각 빌딩별 Validation값

In [123]:
import pandas as pd
import re

# 파일을 열어서 문자열을 읽어옵니다.
with open('best_params_values_0720v2.txt', 'r') as f:
    string = f.read()

# Extract the Iteration and Best Values
pattern = r"Iteration: (\d+),.*?Best Values: ([\d\.]+)"
matches = re.findall(pattern, string, re.DOTALL)

# Convert the matches into a DataFrame
df = pd.DataFrame(matches, columns=["Iteration", "Best Values"])
df["Iteration"] = df["Iteration"].astype(int)
df["Best Values"] = df["Best Values"].astype(float)

print(df)

    Iteration  Best Values
0           1     3.309706
1           2     6.319838
2           3     6.023522
3           4     3.278278
4           5     4.923040
..        ...          ...
95         96     2.608702
96         97     4.062261
97         98     6.098842
98         99     2.525203
99        100     4.073055

[100 rows x 2 columns]


In [124]:
info = pd.read_csv('building_info.csv')
concat = pd.concat([info,df['Best Values']],axis=1)
# Reorder the columns
cols = concat.columns.tolist()  # Convert the column names into a list
cols.insert(2, cols.pop(cols.index('Best Values')))  # Move 'Best Values' to the 3rd position (index 2)

concat = concat[cols]  # Reindex the DataFrame
concat

,건물번호,건물유형,Best Values,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,3.309706,110634.00,39570.00,-,-,-
1,2,건물기타,6.319838,122233.47,99000.00,-,-,-
2,3,건물기타,6.023522,171243.00,113950.00,40,-,-
3,4,건물기타,3.278278,74312.98,34419.62,60,-,-
4,5,건물기타,4.923040,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...,...
95,96,호텔및리조트,2.608702,93314.00,60500.00,-,-,-
96,97,호텔및리조트,4.062261,55144.67,25880.00,-,-,-
97,98,호텔및리조트,6.098842,53578.62,17373.75,-,-,-
98,99,호텔및리조트,2.525203,53499.00,40636.00,-,-,-


In [125]:
import pandas as pd
import re

# 파일을 열어서 문자열을 읽어옵니다.
with open('best_params_values_0721.txt', 'r') as f:
    string = f.read()

# Extract the Iteration and Best Values
pattern = r"Iteration: (\d+),.*?Best Values: ([\d\.]+)"
matches = re.findall(pattern, string, re.DOTALL)

# Convert the matches into a DataFrame
df = pd.DataFrame(matches, columns=["Iteration", "Best Values_day따로"])
df["Iteration"] = df["Iteration"].astype(int)
df["Best Values_day따로"] = df["Best Values_day따로"].astype(float)

print(df)

     Iteration  Best Values_day따로
0            1           3.958431
1            1           5.159142
2            2           7.236492
3            2           6.041059
4            3           5.471286
..         ...                ...
154         96           2.694907
155         97           3.705436
156         98           6.647394
157         99           2.348074
158        100           4.307969

[159 rows x 2 columns]


In [126]:
# Calculate the mean of 'Best Values' for each unique 'Iteration'
df_mean = df.groupby('Iteration')['Best Values_day따로'].mean().reset_index()
df_mean

,Iteration,Best Values_day따로
0,1,4.558786
1,2,6.638776
2,3,5.745692
3,4,3.593410
4,5,4.045336
...,...,...
95,96,2.694907
96,97,3.705436
97,98,6.647394
98,99,2.348074


In [127]:
concat = pd.concat([concat,df_mean['Best Values_day따로']],axis=1)
# Reorder the columns
cols = concat.columns.tolist()  # Convert the column names into a list
cols.insert(3, cols.pop(cols.index('Best Values_day따로')))  # Move 'Best Values' to the 3rd position (index 2)

concat = concat[cols]  # Reindex the DataFrame
concat

,건물번호,건물유형,Best Values,Best Values_day따로,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,3.309706,4.558786,110634.00,39570.00,-,-,-
1,2,건물기타,6.319838,6.638776,122233.47,99000.00,-,-,-
2,3,건물기타,6.023522,5.745692,171243.00,113950.00,40,-,-
3,4,건물기타,3.278278,3.593410,74312.98,34419.62,60,-,-
4,5,건물기타,4.923040,4.045336,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...,...,...
95,96,호텔및리조트,2.608702,2.694907,93314.00,60500.00,-,-,-
96,97,호텔및리조트,4.062261,3.705436,55144.67,25880.00,-,-,-
97,98,호텔및리조트,6.098842,6.647394,53578.62,17373.75,-,-,-
98,99,호텔및리조트,2.525203,2.348074,53499.00,40636.00,-,-,-


In [129]:
concat.to_csv('빌딩별_validation.csv',index=False, encoding = "CP949")